# PTM Flux Model – Exploratory Notebook

This notebook demonstrates the end-to-end pipeline on a toy dataset:
- Load nodes, abundances, and validated edges.
- Load inferred fluxes.
- Build and visualize the flux graph with NetworkX.

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# File paths
nodes_file = "../data/abundances.csv"
edges_file = "../data/validated_edges.txt"
fluxes_file = "../data/fluxes.csv"

In [2]:
# Load node abundances
nodes_df = pd.read_csv(nodes_file)

# Load validated edges
edges = []
with open(edges_file, "r") as f:
    for line in f:
        src, tgt = line.strip().split("\t")
        edges.append((src, tgt))

# Load fluxes
flux_df = pd.read_csv(fluxes_file)

In [3]:
# Build graph
G = nx.DiGraph()

# Add nodes with attributes
for _, row in nodes_df.iterrows():
    node = row["Node_ID"]
    G.add_node(node, abundance_t0=row["t0"], abundance_t1=row["t1"])

# Add edges with flux values
for _, row in flux_df.iterrows():
    G.add_edge(row["source"], row["target"], flux=row["flux"])

In [4]:
# Prepare node sizes (scale by abundance at t1)
node_sizes = [5000 * G.nodes[n]["abundance_t1"] for n in G.nodes]

# Prepare edge widths (scale by flux)
edge_widths = [5 * d["flux"] for _, _, d in G.edges(data=True)]

# Draw graph
pos = nx.spring_layout(G, seed=42)
plt.figure(figsize=(8, 6))

nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color="skyblue")
nx.draw_networkx_labels(G, pos, font_size=10)
nx.draw_networkx_edges(G, pos, width=edge_widths, arrowsize=20, edge_color="gray")

plt.title("Histone Modification Flux Graph (Toy Dataset)")
plt.axis("off")
plt.show()